In [65]:
from kan import KAN
from collections import Counter
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
import torch
from sklearn.utils import shuffle
import random
import numpy as np
torch.set_default_dtype(torch.float32)

In [4]:
# import pandas as pd
# df = pd.read_csv('/tmp/pycharm_project_763/data/trainCPP.csv')
# 
# # 提取序列和标签
# sequences = df['sequence'].tolist()
# y = df['label'].tolist()
# 
# df2= pd.read_csv('/tmp/pycharm_project_763/data/testCPP.csv')
# 
# # 提取序列和标签
# sequences_test = df2['sequence'].tolist()
# y_test = df2['label'].tolist()

In [7]:
# sequences[0],y[0],sequences_test[0], type(y_test[0])

('GWTLNSAGYLLGKINLKALAALAKKIL', 1, 'GKKKKKKKKK', int)

In [105]:
def read_fasta(file_path):
    protein_list=[]
    label_list=[]
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith('>'):
                # 提取标签
                label = line.split('|')[1]
                label_list.append(int(label))
            else:
                # 提取蛋白质序列
                protein_list.append(line)

    return protein_list, label_list

# 示例文件路径
train_path = '/tmp/pycharm_project_763/raw/PBP.txt'
test_path ='/tmp/pycharm_project_763/raw/PBPT.txt'

sequences, y = read_fasta(train_path)
sequences_test, y_test = read_fasta(test_path)

In [106]:
sequences[0],y[0],sequences_test[0], y_test[0] 

('KWKLFKKIEKVGQNIRDGIIKAGPAVAVVGQATQIAK',
 0,
 'KWKLFKKIEKVGQNIRDGIIKAGPAVAVVGQATQIAK',
 0)

In [107]:
def EGAAC(sequences, window=5):
    if window < 1:
        print('Error: the sliding window should be greater than zero' + '\n\n')
        return None

    group = {
        'alphatic': 'GAVLMI',
        'aromatic': 'FYW',
        'positive_charge': 'KRH',
        'negative_charge': 'DE',
        'uncharged': 'STCPNQ'
    }

    groupKeys = group.keys()

    encodings = []
    header = ['#']
    max_len = max(len(seq) for seq in sequences)  # Find the maximum length of sequences
    for w in range(1, max_len - window + 2):
        for g in groupKeys:
            header.append('SW.' + str(w) + '.' + g)
    # encodings.append(header)

    for sequence in sequences:
        code = []
        for j in range(len(sequence) - window + 1):
            subseq = sequence[j:j + window]
            count = Counter(subseq)
            myDict = {}
            for key in groupKeys:
                myDict[key] = sum(count[aa] for aa in group[key] if aa in count)
            for key in groupKeys:
                code.append(myDict[key] / window)
        encodings.append(code)

    return encodings

# 示例蛋白质序列列表

EGAAC_train = EGAAC(sequences, window=5)
max_length = max(len(item) for item in EGAAC_train)

# 进行零填充
train_encodings = np.array([np.pad(item, (0, max_length - len(item)), 'constant', constant_values=(0)) for item in EGAAC_train])
# train_encodings


In [108]:
EGAAC_test = EGAAC(sequences_test, window=5)
# max_length = max(len(item) for item in EGAAC_test)

# 进行零填充
test_encodings = np.array([np.pad(item, (0, max_length - len(item)), 'constant', constant_values=(0)) for item in EGAAC_test])
# test_encodings


In [109]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(train_encodings, y, random_state=42)
X_test, y_test = shuffle(test_encodings, y_test, random_state=42)

In [110]:
X_train=np.array(X_train)
X_test=np.array(X_test)
y_train=np.array(y_train)
y_test=np.array(y_test)

In [111]:
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train).view(-1, 1)
X_test = torch.tensor(X_test)
y_test = torch.tensor(y_test).view(-1, 1)

In [112]:
X_train = torch.tensor(X_train ,dtype=torch.float32)
y_train = torch.tensor(y_train,dtype=torch.float32)
X_test = torch.tensor(X_test,dtype=torch.float32)
y_test = torch.tensor(y_test,dtype=torch.float32)

/home/wangli/anaconda3/envs/transformer-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/home/wangli/anaconda3/envs/transformer-env/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/home/wangli/anaconda3/envs/transformer-env/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing 

In [113]:
X_train.shape,X_test.shape

(torch.Size([500, 465]), torch.Size([532, 465]))

In [114]:
y_test[0],y_test.shape

(tensor([0.]), torch.Size([532, 1]))

In [115]:
dataset = {}
dataset = {
    'train_input': X_train,
    'test_input': X_test,
    'train_label': y_train,
    'test_label': y_test
}


In [116]:
model = KAN(width=[230,5,1], grid=3, k=2, seed=42)

checkpoint directory created: ./model
saving model version 0.0


In [117]:
from sklearn.metrics import roc_auc_score, confusion_matrix, matthews_corrcoef
def train_acc():
    pred=model(X_train)
    with torch.no_grad():
        pred_labels = (pred > 0.5).float() 
        auc = roc_auc_score(y_train.cpu(), pred.cpu())

    # 混淆矩阵计算
        tn, fp, fn, tp = confusion_matrix(y_train.cpu(), pred_labels.cpu()).ravel()

        # Specificity (SP)
        sp = tn / (tn + fp) if (tn + fp) > 0 else 0

        # Sensitivity (SN)
        sn = tp / (tp + fn) if (tp + fn) > 0 else 0

        # Matthews Correlation Coefficient (MCC)
        mcc = matthews_corrcoef(y_train.cpu(), pred_labels.cpu())
    print(f"AUC: {auc:.4f}, SP: {sp:.4f}, MCC: {mcc:.4f}, SN: {sn:.4f}")
    return torch.mean((torch.round(model(X_train)[:, 0]) == y_train[:, 0]).float())

def test_acc():
    return torch.mean((torch.round(model(X_test)[:, 0]) == y_test[:, 0]).float())

# results = model.train(dataset, opt="LBFGS", steps=10, metrics=(train_acc, test_acc)) ,lamb=0.001 lamb_entropy=4.,lamb=0.1,lamb_l1=2.5,
# lamb=0.005 train/fit
results = model.fit(dataset, opt="LBFGS", steps=5, metrics=(train_acc, test_acc));
print(results['train_acc'][-1], results['test_acc'][-1])

| train_loss: 3.20e-01 | test_loss: 4.99e-01 | reg: 0.00e+00 | :  20%|▏| 1/5 [00:02<00:10,  2.56s/it

AUC: 0.9403, SP: 0.8760, MCC: 0.7361, SN: 0.8600


| train_loss: 2.06e-01 | test_loss: 5.06e-01 | reg: 0.00e+00 | :  40%|▍| 2/5 [00:04<00:06,  2.12s/it

AUC: 0.9911, SP: 0.9480, MCC: 0.9040, SN: 0.9560


| train_loss: 1.48e-01 | test_loss: 5.02e-01 | reg: 0.00e+00 | :  60%|▌| 3/5 [00:06<00:03,  1.98s/it

AUC: 0.9955, SP: 0.9760, MCC: 0.9600, SN: 0.9840


| train_loss: 1.17e-01 | test_loss: 5.13e-01 | reg: 0.00e+00 | :  80%|▊| 4/5 [00:08<00:02,  2.06s/it

AUC: 0.9988, SP: 0.9800, MCC: 0.9680, SN: 0.9880


| train_loss: 1.01e-01 | test_loss: 5.19e-01 | reg: 6.00e+01 | : 100%|█| 5/5 [00:12<00:00,  2.56s/it

AUC: 0.9996, SP: 0.9880, MCC: 0.9760, SN: 0.9880
saving model version 0.1
0.9879999756813049 0.7218044996261597


In [118]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50,50)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, alpha=1e-4,
                    solver='sgd', verbose=10, random_state=1,
                    learning_rate_init=.1)
mlp.fit(X_train, y_train)

/home/wangli/anaconda3/envs/transformer-env/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 1, loss = 0.69532209
Iteration 2, loss = 0.64770278
Iteration 3, loss = 0.60567541
Iteration 4, loss = 0.57242321
Iteration 5, loss = 0.55136494
Iteration 6, loss = 0.52428361
Iteration 7, loss = 0.50328327
Iteration 8, loss = 0.48610853
Iteration 9, loss = 0.46990645
Iteration 10, loss = 0.45595826
Iteration 11, loss = 0.44498660
Iteration 12, loss = 0.43319008
Iteration 13, loss = 0.42012422
Iteration 14, loss = 0.41095926
Iteration 15, loss = 0.40296285
Iteration 16, loss = 0.39159482
Iteration 17, loss = 0.38151331
Iteration 18, loss = 0.37473407
Iteration 19, loss = 0.36558455
Iteration 20, loss = 0.35906342
Iteration 21, loss = 0.35236798
Iteration 22, loss = 0.34467775
Iteration 23, loss = 0.33971679
Iteration 24, loss = 0.33577404
Iteration 25, loss = 0.33315645
Iteration 26, loss = 0.32138853
Iteration 27, loss = 0.31811865
Iteration 28, loss = 0.30745227
Iteration 29, loss = 0.30573118
Iteration 30, loss = 0.30011910
Iteration 31, loss = 0.29113946
Iteration 32, los

MLPClassifier(learning_rate_init=0.1, max_iter=300, random_state=1,
              solver='sgd', verbose=10)

In [119]:
predictions_train = mlp.predict(X_train)
predictions_test = mlp.predict(X_test)

# 计算准确率
train_accuracy = accuracy_score(y_train, predictions_train)
test_accuracy = accuracy_score(y_test, predictions_test)

print(f'Train Accuracy: {train_accuracy:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

Train Accuracy: 0.9920
Test Accuracy: 0.7237
